# Lab 3-1: REINFORCE
    In this lab, you need to implement a REINFORCE algorithm with Tensorflow and solve OpenAI Gym CartPole-v0

In [1]:
from cartpole_env import *

import numpy as np
import tensorflow as tf

from collections import namedtuple, deque

# Define the data structure of experience
Experience = namedtuple('Experience', 'state action reward next_state done')

## Implement ```discount``` function to compute discounted reward

In [2]:
def discount(rewards, gamma):
    '''
    param rewards: a rewards numpy array
    param gamma: discount factor
    '''
    discounted_rewards = np.zeros_like(rewards)
    
    # TODO: Calculate discounted rewards
    discounted_rewards = rewards
    s = 0
    for i in reversed(range(len(rewards))):
        #
        # reward = [1, 2, 3]
        # discounted_rewards = [
        #    1 + gamma * 2 + gamma * gamma * 3, 
        #    2 + gamma * 3,
        #    3 ]
        #
        
        s = s * gamma + rewards[i]
        discounted_rewards[i] = s

    return discounted_rewards

## Implement ```do_rollout``` function to collect rollout

In [3]:
def do_rollout(env, policy, render=False):
    '''
    Collect a rollout from env with policy
    
    param env: RL Environment
    param policy: a function parameterized by environment state, return a action
    return a list of (state, action, reward, next_state, done)
    '''
    # Initialize done as False
    done = False
    
    # Reset the environment and get the initial state
    state = env.reset()
    
    # Empty list
    rollout = []
    
    while not done:
        action = policy(state)
        next_state, reward, done, info = env.step(action)
        
        # Render the environment (slow)
        if render:
            env.render()
        
        rollout.append(Experience(state, action, reward, next_state, done))
        state = next_state
        
    return rollout

## Implement ```ReinforceAgent``` following ```TODO```

In [4]:
class ReinforceAgent(object):
    def __init__(self, sess, n_states, n_actions, n_hiddens, lr, gamma):
        '''
        param sess: tf session
        param n_states: dim of states
        param n_actions: dim of actions space
        param n_hiddens: dim of hidden state
        '''
        self.sess = sess
        self.n_states = n_states
        self.n_actions = n_actions
        self.n_hiddens = n_hiddens
        
        # Learning rate
        self.lr = lr
        
        # Discount factor
        self.gamma = gamma
       
        self.state = tf.placeholder(shape=[None, n_states], dtype=tf.float32)
        self.value = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action = tf.placeholder(shape=[None], dtype=tf.int32)
        
        # TODO: Declare 1-st hidden layer
        # Define a fully-connected layer with:
        # input = self.state
        # n_units = self.n_hiddens
        # activation = relu
        # weight_initializer = random_normal(0.0, 0.1)
        # bias_initializer = constant (0.1)
        self.h = tf.layers.dense(
            inputs=self.state,
            units=n_hiddens,   
            activation=tf.nn.relu,
            kernel_initializer=tf.random_normal_initializer(0., .1), 
            bias_initializer=tf.constant_initializer(0.1), 
            name='h')
        
        # Declare 1-st hidden layer
        # Define a fully-connected layer with:
        # input = 1-st hidden layer
        # n_units = self.n_actions
        # activation = relu
        # weight_initializer = random_normal(0.0, 0.1)
        # bias_initializer = constant (0.1)
        self.policy = tf.layers.dense(
            inputs=self.h,
            units=n_actions,    
            activation=tf.nn.softmax,  
            kernel_initializer=tf.random_normal_initializer(0., .1), 
            bias_initializer=tf.constant_initializer(0.1),
            name='policy')
        
        # TODO: negative log probability
        neg_log = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=self.action, logits=self.policy)
        
        # TODO: policy gradient loss function
        self.loss = tf.reduce_mean(neg_log * self.value)
        
        # Optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        self.train_op = self.optimizer.minimize(self.loss)
        
    def layer(self, inputs, n_input, n_units, activation=tf.nn.relu):
        weights = tf.Variable(
            tf.random_normal([n_input, n_units], mean=0.0, stddev=0.1), name="weights")
        bias = tf.Variable(tf.constant(0.1, shape=[n_units]))
        outputs = activation(tf.matmul(inputs, weights) + bias) 
        
        return outputs
    
    def act(self, s):
        '''
        param s: a np.ndarray with shape [n_batches, n_states]
        return a batch of actions with shape [n_batches,]
        '''
        # TODO: Softmax stochastic policy
        policy = sess.run(self.policy, feed_dict={self.state: s})
        action = np.random.choice(self.n_actions, 1, p=policy.reshape(-1))
        
        return action
    
    def train(self, rollout):
        '''
        param rollout: a list o
        '''
        states = np.array([np.asarray(e.state) for e in rollout ])
        actions = np.squeeze(np.array([ e.action for e in rollout ]))
        rewards = np.array([ e.reward for e in rollout ])
        discounted_rewards = discount(rewards, gamma=self.gamma)
        
        self.sess.run(self.train_op, feed_dict={self.state: states,
                                                self.action: actions,
                                                self.value: discounted_rewards})
   
        

In [5]:
LR = 0.001
GAMMA = 0.99

sess = tf.InteractiveSession()
env = CartpoleEnvironment()
agent = ReinforceAgent(sess=sess, 
                       n_states=env.observation_space.shape[0],
                       n_actions=env.action_space.n,
                       n_hiddens=20,
                       lr=LR,
                       gamma=GAMMA)
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
def policy(s):
    return agent.act([s])[0]

In [7]:
def calculate_episode_reward(rollout):
    rewards = [ e.reward for e in rollout ]
    return sum(rewards)

In [ ]:
def eval_history_reward(history):
    arr = np.asarray(history)
    return arr.mean()

In [ ]:
MAX_ITERATIONS = 100000

episode_reward = 0.0
history_episode_rewards = deque(maxlen=100)

plot_history_episode_rewards = []

for iter in range(MAX_ITERATIONS):
    rollout = do_rollout(env=env, policy=policy, render=False)
    agent.train(rollout=rollout)
    
    episode_reward = calculate_episode_reward(rollout)
    history_episode_rewards.append(episode_reward)
    plot_history_episode_rewards.append(episode_reward)
    mean_rewards = eval_history_reward(history_episode_rewards)
    
    print('Episode %d: Reward = %f, Mean reward (over %d episodes) = %f' % (iter, 
                                                                            episode_reward,
                                                                            len(history_episode_rewards),
                                                                            mean_rewards))
    if mean_rewards > 195.0:
        print('Pass')
        break

Episode 0: Reward = -9.000000, Mean reward (over 1 episodes) = -9.000000
Episode 1: Reward = 4.000000, Mean reward (over 2 episodes) = -2.500000
Episode 2: Reward = 2.000000, Mean reward (over 3 episodes) = -1.000000
Episode 3: Reward = 14.000000, Mean reward (over 4 episodes) = 2.750000
Episode 4: Reward = 9.000000, Mean reward (over 5 episodes) = 4.000000
Episode 5: Reward = 8.000000, Mean reward (over 6 episodes) = 4.666667
Episode 6: Reward = -1.000000, Mean reward (over 7 episodes) = 3.857143
Episode 7: Reward = -2.000000, Mean reward (over 8 episodes) = 3.125000
Episode 8: Reward = -10.000000, Mean reward (over 9 episodes) = 1.666667
Episode 9: Reward = 23.000000, Mean reward (over 10 episodes) = 3.800000
Episode 10: Reward = 3.000000, Mean reward (over 11 episodes) = 3.727273
Episode 11: Reward = -10.000000, Mean reward (over 12 episodes) = 2.583333
Episode 12: Reward = 27.000000, Mean reward (over 13 episodes) = 4.461538
Episode 13: Reward = 26.000000, Mean reward (over 14 epis

Episode 116: Reward = 30.000000, Mean reward (over 100 episodes) = 2.230000
Episode 117: Reward = 17.000000, Mean reward (over 100 episodes) = 2.460000
Episode 118: Reward = 0.000000, Mean reward (over 100 episodes) = 2.510000
Episode 119: Reward = 20.000000, Mean reward (over 100 episodes) = 2.730000
Episode 120: Reward = -2.000000, Mean reward (over 100 episodes) = 2.820000
Episode 121: Reward = -9.000000, Mean reward (over 100 episodes) = 2.530000
Episode 122: Reward = 5.000000, Mean reward (over 100 episodes) = 2.670000
Episode 123: Reward = 26.000000, Mean reward (over 100 episodes) = 3.000000
Episode 124: Reward = 14.000000, Mean reward (over 100 episodes) = 3.250000
Episode 125: Reward = -8.000000, Mean reward (over 100 episodes) = 3.280000
Episode 126: Reward = -2.000000, Mean reward (over 100 episodes) = 3.190000
Episode 127: Reward = 30.000000, Mean reward (over 100 episodes) = 3.490000
Episode 128: Reward = -11.000000, Mean reward (over 100 episodes) = 3.490000
Episode 129: 

Episode 231: Reward = 48.000000, Mean reward (over 100 episodes) = 5.910000
Episode 232: Reward = 4.000000, Mean reward (over 100 episodes) = 6.040000
Episode 233: Reward = -8.000000, Mean reward (over 100 episodes) = 5.650000
Episode 234: Reward = 42.000000, Mean reward (over 100 episodes) = 6.190000
Episode 235: Reward = -1.000000, Mean reward (over 100 episodes) = 6.170000
Episode 236: Reward = -7.000000, Mean reward (over 100 episodes) = 6.130000
Episode 237: Reward = 1.000000, Mean reward (over 100 episodes) = 6.220000
Episode 238: Reward = 35.000000, Mean reward (over 100 episodes) = 6.690000
Episode 239: Reward = -7.000000, Mean reward (over 100 episodes) = 6.550000
Episode 240: Reward = 15.000000, Mean reward (over 100 episodes) = 6.770000
Episode 241: Reward = 13.000000, Mean reward (over 100 episodes) = 6.880000
Episode 242: Reward = 29.000000, Mean reward (over 100 episodes) = 6.890000
Episode 243: Reward = 20.000000, Mean reward (over 100 episodes) = 7.070000
Episode 244: R

Episode 344: Reward = 29.000000, Mean reward (over 100 episodes) = 13.680000
Episode 345: Reward = 3.000000, Mean reward (over 100 episodes) = 13.570000
Episode 346: Reward = 7.000000, Mean reward (over 100 episodes) = 13.370000
Episode 347: Reward = -10.000000, Mean reward (over 100 episodes) = 13.140000
Episode 348: Reward = 10.000000, Mean reward (over 100 episodes) = 13.130000
Episode 349: Reward = -4.000000, Mean reward (over 100 episodes) = 13.100000
Episode 350: Reward = 49.000000, Mean reward (over 100 episodes) = 13.500000
Episode 351: Reward = 8.000000, Mean reward (over 100 episodes) = 13.390000
Episode 352: Reward = -8.000000, Mean reward (over 100 episodes) = 12.950000
Episode 353: Reward = -4.000000, Mean reward (over 100 episodes) = 12.750000
Episode 354: Reward = 4.000000, Mean reward (over 100 episodes) = 12.740000
Episode 355: Reward = 19.000000, Mean reward (over 100 episodes) = 12.860000
Episode 356: Reward = 8.000000, Mean reward (over 100 episodes) = 12.950000
Epi

Episode 451: Reward = 60.000000, Mean reward (over 100 episodes) = 26.290000
Episode 452: Reward = 29.000000, Mean reward (over 100 episodes) = 26.660000
Episode 453: Reward = 2.000000, Mean reward (over 100 episodes) = 26.720000
Episode 454: Reward = 5.000000, Mean reward (over 100 episodes) = 26.730000
Episode 455: Reward = 30.000000, Mean reward (over 100 episodes) = 26.840000
Episode 456: Reward = 56.000000, Mean reward (over 100 episodes) = 27.320000
Episode 457: Reward = 18.000000, Mean reward (over 100 episodes) = 26.930000
Episode 458: Reward = 40.000000, Mean reward (over 100 episodes) = 27.370000
Episode 459: Reward = 34.000000, Mean reward (over 100 episodes) = 27.570000
Episode 460: Reward = 24.000000, Mean reward (over 100 episodes) = 27.670000
Episode 461: Reward = 70.000000, Mean reward (over 100 episodes) = 28.190000
Episode 462: Reward = 23.000000, Mean reward (over 100 episodes) = 28.460000
Episode 463: Reward = 7.000000, Mean reward (over 100 episodes) = 28.370000
Ep

...

Episode 4943: Reward = 200.000000, Mean reward (over 100 episodes) = 177.680000
Episode 4944: Reward = 119.000000, Mean reward (over 100 episodes) = 176.870000
Episode 4945: Reward = 200.000000, Mean reward (over 100 episodes) = 176.870000
Episode 4946: Reward = 133.000000, Mean reward (over 100 episodes) = 176.960000
Episode 4947: Reward = 158.000000, Mean reward (over 100 episodes) = 176.540000
Episode 4948: Reward = 200.000000, Mean reward (over 100 episodes) = 176.810000
Episode 4949: Reward = 168.000000, Mean reward (over 100 episodes) = 177.060000
Episode 4950: Reward = 174.000000, Mean reward (over 100 episodes) = 176.800000
Episode 4951: Reward = 200.000000, Mean reward (over 100 episodes) = 177.670000
Episode 4952: Reward = 124.000000, Mean reward (over 100 episodes) = 176.910000
Episode 4953: Reward = 134.000000, Mean reward (over 100 episodes) = 176.650000
Episode 4954: Reward = 123.000000, Mean reward (over 100 episodes) = 175.880000
Episode 4955: Reward = 148.000000, Mean 

Episode 5047: Reward = 176.000000, Mean reward (over 100 episodes) = 168.930000
Episode 5048: Reward = 123.000000, Mean reward (over 100 episodes) = 168.160000
Episode 5049: Reward = 200.000000, Mean reward (over 100 episodes) = 168.480000
Episode 5050: Reward = 200.000000, Mean reward (over 100 episodes) = 168.740000
Episode 5051: Reward = 110.000000, Mean reward (over 100 episodes) = 167.840000
Episode 5052: Reward = 119.000000, Mean reward (over 100 episodes) = 167.790000
Episode 5053: Reward = 140.000000, Mean reward (over 100 episodes) = 167.850000
Episode 5054: Reward = 135.000000, Mean reward (over 100 episodes) = 167.970000
Episode 5055: Reward = 200.000000, Mean reward (over 100 episodes) = 168.490000
Episode 5056: Reward = 165.000000, Mean reward (over 100 episodes) = 168.140000
Episode 5057: Reward = 144.000000, Mean reward (over 100 episodes) = 167.580000
Episode 5058: Reward = 153.000000, Mean reward (over 100 episodes) = 167.830000
Episode 5059: Reward = 153.000000, Mean 

Episode 5150: Reward = 200.000000, Mean reward (over 100 episodes) = 164.000000
Episode 5151: Reward = 107.000000, Mean reward (over 100 episodes) = 163.970000
Episode 5152: Reward = 127.000000, Mean reward (over 100 episodes) = 164.050000
Episode 5153: Reward = 125.000000, Mean reward (over 100 episodes) = 163.900000
Episode 5154: Reward = 200.000000, Mean reward (over 100 episodes) = 164.550000
Episode 5155: Reward = 130.000000, Mean reward (over 100 episodes) = 163.850000
Episode 5156: Reward = 200.000000, Mean reward (over 100 episodes) = 164.200000
Episode 5157: Reward = 200.000000, Mean reward (over 100 episodes) = 164.760000
Episode 5158: Reward = 200.000000, Mean reward (over 100 episodes) = 165.230000
Episode 5159: Reward = 140.000000, Mean reward (over 100 episodes) = 165.100000
Episode 5160: Reward = 169.000000, Mean reward (over 100 episodes) = 164.790000
Episode 5161: Reward = 154.000000, Mean reward (over 100 episodes) = 165.020000
Episode 5162: Reward = 200.000000, Mean 

Episode 5253: Reward = 176.000000, Mean reward (over 100 episodes) = 161.850000
Episode 5254: Reward = 150.000000, Mean reward (over 100 episodes) = 161.350000
Episode 5255: Reward = 120.000000, Mean reward (over 100 episodes) = 161.250000
Episode 5256: Reward = 200.000000, Mean reward (over 100 episodes) = 161.250000
Episode 5257: Reward = 175.000000, Mean reward (over 100 episodes) = 161.000000
Episode 5258: Reward = 200.000000, Mean reward (over 100 episodes) = 161.000000
Episode 5259: Reward = 200.000000, Mean reward (over 100 episodes) = 161.600000
Episode 5260: Reward = 121.000000, Mean reward (over 100 episodes) = 161.120000
Episode 5261: Reward = 172.000000, Mean reward (over 100 episodes) = 161.300000
Episode 5262: Reward = 121.000000, Mean reward (over 100 episodes) = 160.510000
Episode 5263: Reward = 124.000000, Mean reward (over 100 episodes) = 159.750000
Episode 5264: Reward = 132.000000, Mean reward (over 100 episodes) = 159.490000
Episode 5265: Reward = 200.000000, Mean 

Episode 5357: Reward = 134.000000, Mean reward (over 100 episodes) = 162.790000
Episode 5358: Reward = 162.000000, Mean reward (over 100 episodes) = 162.410000
Episode 5359: Reward = 125.000000, Mean reward (over 100 episodes) = 161.660000
Episode 5360: Reward = 146.000000, Mean reward (over 100 episodes) = 161.910000
Episode 5361: Reward = 146.000000, Mean reward (over 100 episodes) = 161.650000
Episode 5362: Reward = 168.000000, Mean reward (over 100 episodes) = 162.120000
Episode 5363: Reward = 154.000000, Mean reward (over 100 episodes) = 162.420000
Episode 5364: Reward = 162.000000, Mean reward (over 100 episodes) = 162.720000
Episode 5365: Reward = 161.000000, Mean reward (over 100 episodes) = 162.330000
Episode 5366: Reward = 200.000000, Mean reward (over 100 episodes) = 162.730000


In [ ]:
import matplotlib.pyplot as plt

def plot(x, y, name):
    fig, ax = plt.subplots()
    ax.plot(x, y)
    
    ax.set(xlabel='Episode', ylabel='Reward', title=name)
    ax.grid()

    fig.savefig("%s.png" % name)
    plt.show()
    
plot(range(episode), plot_history_episode_rewards, 'REINFORCE')

In [ ]:
while True:
    do_rollout(env=env, policy=policy, render=True)